## TensorBoardサンプルコード

* tensorflow==1.13.1

on_epoch_endイベント時にTensorBoardにクラスごとのprecision, recall, f1-scoreを出力する。

### 簡易なモデルを記述（fashion-mnist）

In [2]:
import datetime
import numpy as np

import tensorflow as tf
from tensorflow import keras
from sklearn import model_selection
from tensorflow.keras import layers

# データの用意
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0

# testデータの1/10をvalidationデータに使う
test_images, val_images, test_labels, val_labels = model_selection.train_test_split(
    test_images, test_labels, test_size=0.1, stratify=test_labels)

# 単純な推定器
model = tf.keras.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

## ラベル一覧

In [3]:
LABELS = [
    'T-shirt/top',
    'Trouser',
    'Pullover',
    'Dress',
    'Coat',
    'Sandal',
    'Shirt',
    'Sneaker',
    'Bag',
    'Ankle boot',
]

## callbacksの記述

tf.keras.callbacks.TensorBoard を継承して on_epoch_end 時にsummaryに書き込む処理を実行する。

In [4]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

class MyCallback(tf.keras.callbacks.TensorBoard):
    
    def __init__(self, test_images, test_labels, **kwargs):
        super().__init__(**kwargs)
        self.test_images = test_images
        self.test_labels = test_labels

    def on_epoch_end(self, epoch, logs={}):
        super().on_epoch_end(epoch, logs)

        # predict実行
        y_pred = np.argmax(model.predict(test_images), axis=1).astype(np.int8)
        
        # precision/recall/f-score取得
        report = classification_report(test_labels, y_pred, output_dict=True)

        # 指定指標のValuesを生成
        def get_values(key):
            return np.array([tf.Summary.Value(
                tag='%s/%s' % (key, LABELS[label]),
                simple_value=report[str(label)][key])
            for label in range(10)])
        
        values = np.array([
            get_values('precision'), 
            get_values('recall'),
            get_values('f1-score'),
        ]).flatten()
        
        # summaryの設定
        summary = tf.Summary(value=values)
        self.writer.add_summary(summary, epoch)

## callbacksを指定してfit

自作のcallbackを指定してfitを実行。

In [ ]:
my_callback = MyCallback(
    test_images, test_labels,
    log_dir='logs')

model.fit(
    train_images,
    train_labels,
    epochs=50,
    callbacks=[my_callback])